In [141]:
import os
from parsons import VAN, Redshift, Table
from datetime import datetime, timedelta
from canalespy import logger, setup_environment

In [142]:
setup_environment()
rs = Redshift()
van = VAN(api_key='f49f4c62-439c-362d-eae2-e6b32ccb8431', db='EveryAction')

In [3]:
LOG_TABLE = 'txop_newmode.calls_to_ea_log'
SOURCE_TABLE = 'txop_newmode.call_summary_to_ea_test'

In [4]:
if not rs.table_exists(LOG_TABLE): #if Log Table does not already exist, create it
    # standard sql
    sql = "(outreach_id int, vanid int, date_created timestamp, sr_load varchar(100), event_load varchar(100));"
    rs.query(f"create table {LOG_TABLE} {sql}")
    logger.info(f"Creating {LOG_TABLE}...")

In [6]:
source_tbl = rs.query(f'''
  select * from
    (select *, row_number() over(partition by outreach_id order by action_date desc) as dup
    from {SOURCE_TABLE}) a
  left join {LOG_TABLE} l using(outreach_id)
  where
  l.outreach_id is null
  and dup=1
  ;
  '''
)

In [50]:
source_tbl

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank,dup,vanid,date_created,sr_load,event_load
9247520,2021-01-24 00:29:34,Damaris,Gonzalez,gonzalezmrs34@gmail.com,7138551968,77036,https://organizetexas.org/immigration-2021/,695591,MDW,completed,User hung up phone,1,1,None,None,None,None
9260078,2021-01-26 05:04:02,Alfonso,Ayala,alfonso.ayala3@gmail.com,3238734535,95112,https://organizetexas.org/immigration-2021/?emci=8f1950d0-7a5f-eb11-a607-00155d43c992&emdi=1d896589-8e5f-eb11-a607-00155d43c992&ceid=97003,695591,MDW,completed,User hung up phone,1,1,None,None,None,None
9197954,2021-01-21 01:33:28,Adelina,Cáceres,prosperitygroup7@gmail.com,8326925745,77093,https://organizetexas.org/immigration-2021/,695591,MDW,completed,User hung up phone,1,1,None,None,None,None
10218200,2021-03-23 21:40:33,Gina,Villarreal,ginalvilla1972@gmail.com,2103287149,78226,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,User hung up phone,1,1,None,None,None,None
9247533,2021-01-24 00:32:19,Damaris,Gonzalez,gonzalezmrs34@gmail.com,7138551968,77036,https://organizetexas.org/immigration-2021/,695591,MDW,completed,User hung up phone,3,1,None,None,None,None


In [82]:
loaded = [['outreach_id', 'vanid', 'datecreated','sr_load', 'signup_id']]

In [84]:
type(loaded[0])

list

In [144]:
loaded = [['first_name', 'vanId']]

people = [
    {'first_name':'Damaris'
      ,'last_name':'Gonzalez'
      ,'email':'gonzalezmrs34@gmail.com'
      ,'phone':7138551968
      ,'zip':77036
    },
     {'first_name':'erica'
      ,'last_name':'faulkenberry'
      ,'email':'efaulkenberry@gmail.com'
      ,'phone':2144574671
      ,'zip':78721
    },
]

for ppl in people:

    try:
       ea = van.find_person(first_name=ppl['first_name']
                                  ,last_name=ppl['last_name']
                                  ,email=ppl['email']
                                  ,phone=ppl['phone']
                                  ,zip=ppl['zip'])
    except Exception as e:
        logger.info(f"Issue finding in VAN! {str(e)}")
        ea['vanId'] = e
    
    try:
        test = ea['vanId']
        print(test)
    except:
        test = ea
        print(ea)

people INFO Finding Damaris Gonzalez.
people INFO Finding erica faulkenberry.


101547194


INFO Issue finding in VAN! HTTP error occurred (404): Not Found, json: {'vanId': None, 'status': 'Unmatched'}


HTTP error occurred (404): Not Found, json: {'vanId': None, 'status': 'Unmatched'}


In [ ]:
#     van = [ppl['first_name'], str(ea['vanId'])]
#     #['vanId'] if ea['vanId'] is not None else str(ea)
#     #  ppl[ea] if ppl[ea] else str(ea['vanId'])
#     loaded.append(van)
#     print(loaded)

for ppl in source_tbl:
    ea = van.find_person(first_name=ppl['first_name']
                          ,last_name=ppl['last_name']
                          ,email=ppl['email']
                          ,phone=ppl['phone']
                          ,zip=ppl['zip'])

    action_date_transform = datetime.strptime(ppl['action_date'], '%Y-%m-%d %H:%M:%S').isoformat()

    print(ea['firstName'],action_date_transform,ea['vanId'])

In [71]:
type(source_tbl['survey_response_id'][0])

int

In [35]:
type(source_tbl['action_date'][0])

str

In [43]:
source_tbl['action_date'][0]

'2021-01-24 00:29:34'

In [44]:
action_date_transform = datetime.strptime(source_tbl['action_date'][0], '%Y-%m-%d %H:%M:%S').isoformat()

In [73]:
action_date_transform

'2021-03-23T21:46:21'

In [8]:
logger.info(f"{source_tbl.num_rows} rows found.")

INFO 16 rows found.
INFO:canalespy:16 rows found.


In [9]:
if source_tbl.num_rows > 0:
      logger.info(f"{source_tbl.num_rows} new records found. Inserting into table")
      loaded = [['outreach_id', 'vanid', 'datecreated','sr_load', 'event_load']] #column names for our eventual Parsons Table of what we've loaded

      for ppl in source_tbl:
        logger.info(f"Working on outreach_id={ppl['outreach_id']}")
        
        # find contacts in ea
        ea = van.find_person(first_name=ppl['first_name'],
                    last_name=ppl['last_name'],
                    email=ppl['email'],
                    phone=ppl['phone'],
                    zip=ppl['zip'])

        if isinstance(ea, tuple):
          logger.info("Issue loading to VAN!")
          vanids = [ppl['vanid'], 0, TIMESTAMP, 'Error', 'Error', 'Error', 'Error']
          continue

        logger.info(f"Found in EveryAction as {ea['vanId']}...")

INFO 16 new records found. Inserting into table
INFO:canalespy:16 new records found. Inserting into table
INFO Working on outreach_id=9247520
INFO:canalespy:Working on outreach_id=9247520
people INFO Finding Damaris  Gonzalez.
INFO:parsons.ngpvan.people:Finding Damaris  Gonzalez.
INFO Found in EveryAction as 101547194...
INFO:canalespy:Found in EveryAction as 101547194...
INFO Working on outreach_id=9260078
INFO:canalespy:Working on outreach_id=9260078
people INFO Finding Alfonso Ayala.
INFO:parsons.ngpvan.people:Finding Alfonso Ayala.
INFO Found in EveryAction as 101925984...
INFO:canalespy:Found in EveryAction as 101925984...
INFO Working on outreach_id=9197954
INFO:canalespy:Working on outreach_id=9197954
people INFO Finding Adelina  Cáceres .
INFO:parsons.ngpvan.people:Finding Adelina  Cáceres .
INFO Found in EveryAction as 102020619...
INFO:canalespy:Found in EveryAction as 102020619...
INFO Working on outreach_id=10218200
INFO:canalespy:Working on outreach_id=10218200
people INFO

In [81]:
mdw = van.get_event(event_id=750016899,expand_fields=['locations', 'codes', 'shifts', 'roles', 'notes', 'financialProgram', 'ticketCategories', 'voterRegistrationBatches'])

events INFO Found event 750016899.
INFO:parsons.ngpvan.events:Found event 750016899.


In [82]:
mdw

{'eventId': 750016899,
 'name': 'One on One',
 'shortName': '1on1',
 'description': None,
 'startDate': '2021-02-26T20:00:00-05:00',
 'endDate': '2021-02-26T21:00:00-05:00',
 'eventType': {'eventTypeId': 128552, 'name': 'One-on-Ones'},
 'isOnlyEditableByCreatingUser': False,
 'isPubliclyViewable': None,
 'locations': [],
 'codes': [],
 'notes': [],
 'shifts': [{'eventShiftId': 33212,
   'name': 'Single Shift',
   'startTime': '2021-02-26T20:00:00-05:00',
   'endTime': '2021-02-26T21:00:00-05:00'}],
 'roles': [{'roleId': 100525,
   'name': 'Action leader',
   'isEventLead': True,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100526,
   'name': 'Attendee',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100529,
   'name': 'Doorknocking',
   'isEventLead': True,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100531,
   'name': 'Fliers',
   'isEventLead': True,
   'min': None,
   'max': None,
   'goal': None},
  {'rol

In [104]:
#event = get_events(event_type_ids=None, rep_event_id=None, starting_after=None, expand_fields=['locations', 'codes', 'shifts', 'roles', 'notes', 'financialProgram', 'ticketCategories', 'onlineForms'])

e = van.get_event(event_id=750015212)



events INFO Found event 750015212.
INFO:parsons.ngpvan.events:Found event 750015212.


In [105]:
e

{'eventId': 750015212,
 'name': 'Member Doing Work',
 'shortName': 'MDW',
 'description': None,
 'startDate': '2021-02-13T20:00:00-05:00',
 'endDate': '2021-02-13T20:30:00-05:00',
 'eventType': {'eventTypeId': 128562, 'name': 'Member Doing Work'},
 'isOnlyEditableByCreatingUser': False,
 'isPubliclyViewable': None,
 'locations': [],
 'codes': [],
 'notes': [],
 'shifts': [{'eventShiftId': 31519,
   'name': 'Single Shift',
   'startTime': '2021-02-13T20:00:00-05:00',
   'endTime': '2021-02-13T20:30:00-05:00'}],
 'roles': [{'roleId': 100525,
   'name': 'Action leader',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100526,
   'name': 'Attendee',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100527,
   'name': 'Chant Leader',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100528,
   'name': 'Data Entry',
   'isEventLead': False,
   'min': None,
   'max': None,
   'go

In [46]:
mwd_2 = van.get_events(event_type_ids=128562,starting_after='2021-03-23', starting_before="2021-03-25")

events INFO Found 1 events.
INFO:parsons.ngpvan.events:Found 1 events.


In [47]:
mwd_2

eventId,name,shortName,description,startDate,endDate,eventType,isOnlyEditableByCreatingUser,isPubliclyViewable,locations,codes,notes,shifts,roles,ticketCategories,voterRegistrationBatches,createdDate,districtFieldValue,financialProgram,isActive,dotNetTimeZoneId
750015251,Member Doing Work,MDW,None,2021-03-24T20:00:00-04:00,2021-03-24T20:30:00-04:00,"{'eventTypeId': 128562, 'name': 'Member Doing Work'}",False,None,[],[],[],"[{'eventShiftId': 31558, 'name': 'Single Shift', 'startTime': '2021-03-24T20:00:00-04:00', 'endTime': '2021-03-24T20:30:00-04:00'}]","[{'roleId': 100525, 'name': 'Action leader', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100526, 'name': 'Attendee', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100527, 'name': 'Chant Leader', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100528, 'name': 'Data Entry', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100529, 'name': 'Doorknocking', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100530, 'name': 'Driver', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100531, 'name': 'Fliers', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100532, 'name': 'Fundraiser', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100533, 'name': 'House meeting host', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100534, 'name': 'Media Spokesperson', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100535, 'name': 'Meeting facilatator', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100536, 'name': 'Member mover', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100537, 'name': 'Office support', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100538, 'name': 'Online networking', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100539, 'name': 'Petitioning', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100540, 'name': 'Phone banking', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100541, 'name': 'Talked to Politician/Elected Official', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100542, 'name': 'Signs up members/Collects dues', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100543, 'name': 'Talk to Civic Organizations', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100544, 'name': 'Talk to faith leaders', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100545, 'name': 'Voter captain', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100563, 'name': 'Registration/Sign-In', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 100564, 'name': 'Secured Venue', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 110923, 'name': 'Meeting Preparation', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 120729, 'name': 'Services Outreach', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}, {'roleId': 132067, 'name': 'Ally Work', 'isEventLead': False, 'min': None, 'max': None, 'goal': None}]",[],None,2020-04-16T17:46:00Z,None,None,True,Eastern Standard Time


In [48]:
location = mwd_2['locations']

In [49]:
location

[[]]

In [163]:
result = mwd_2['eventId']

In [164]:
result[0]

750015251

In [20]:
action_date = source_tbl['action_date']

In [177]:
action_date

['2021-03-23 17:38:25']

In [204]:
action_date_2 = str(datetime.strptime(action_date[0], '%Y-%m-%d %H:%M:%S'))

In [205]:
action_date_2

'2021-03-23 17:38:25'

In [246]:
action_date_3 = datetime.strptime(action_date[0], '%Y-%m-%d %H:%M:%S')

In [248]:
type(action_date_3)

datetime.datetime

In [21]:
for ppl in source_tbl:
    a_date_str = str(datetime.strptime(ppl['action_date'], '%Y-%m-%d %H:%M:%S').date())

In [229]:
a_date_str

'2021-03-23'

In [22]:
for ppl in source_tbl:
    a_date = datetime.strptime(ppl['action_date'], '%Y-%m-%d %H:%M:%S').date()

datetime.date

In [23]:
action_date_subtract= a_date - timedelta(days=1)

In [280]:
action_date_subtract

datetime.datetime(2021, 3, 22, 17, 38, 25)

In [268]:
action_date_3

datetime.datetime(2021, 3, 23, 17, 38, 25)

In [24]:
action_date_add = str(a_date + timedelta(days=1))

In [284]:
action_date_add

'2021-03-24'

In [271]:
action_date_3

datetime.datetime(2021, 3, 23, 17, 38, 25)

In [257]:
d = datetime.today() - timedelta(days=1)

In [258]:
d

datetime.datetime(2021, 3, 23, 20, 4, 51, 385481)

In [168]:
unique_sr_id = {row['survey_response_id'] for row in source_tbl}

In [169]:
unique_sr_id

{914328}

In [25]:
action_date_add_str = str(action_date_add)

In [26]:
action_date_subtract_str = str(action_date_subtract)

In [58]:
find_mdw_event = van.get_events(event_type_ids=128562,starting_after={action_date_subtract_str}, starting_before={action_date_add_str})

events INFO Found 1 events.
INFO:parsons.ngpvan.events:Found 1 events.


In [67]:
event_id = find_mdw_event['eventId'][0]

In [68]:
type(event_id)

int

In [29]:
shifts = find_mdw_event['shifts'][0][0]

In [30]:
shifts

{'eventShiftId': 31557,
 'name': 'Single Shift',
 'startTime': '2021-03-23T20:00:00-04:00',
 'endTime': '2021-03-23T20:30:00-04:00'}

In [304]:
shifts[0]

[{'eventShiftId': 31557,
  'name': 'Single Shift',
  'startTime': '2021-03-23T20:00:00-04:00',
  'endTime': '2021-03-23T20:30:00-04:00'}]

In [298]:
shifts[0][0]

{'eventShiftId': 31557,
 'name': 'Single Shift',
 'startTime': '2021-03-23T20:00:00-04:00',
 'endTime': '2021-03-23T20:30:00-04:00'}

In [300]:
type(shifts[0][0])

dict

In [309]:
shifts1 = {**shifts[0][0]}

In [306]:
shifts1

{'eventShiftId': 31557,
 'name': 'Single Shift',
 'startTime': '2021-03-23T20:00:00-04:00',
 'endTime': '2021-03-23T20:30:00-04:00'}

In [310]:
print([(shifts, shifts1[shifts]) for shifts in shifts1])

[('eventShiftId', 31557), ('name', 'Single Shift'), ('startTime', '2021-03-23T20:00:00-04:00'), ('endTime', '2021-03-23T20:30:00-04:00')]


In [311]:
for key, value in shifts1.items():
    print (key, value)

eventShiftId 31557
name Single Shift
startTime 2021-03-23T20:00:00-04:00
endTime 2021-03-23T20:30:00-04:00


In [312]:
for i in enumerate(shifts1.items()):
    print(i)

(0, ('eventShiftId', 31557))
(1, ('name', 'Single Shift'))
(2, ('startTime', '2021-03-23T20:00:00-04:00'))
(3, ('endTime', '2021-03-23T20:30:00-04:00'))


In [313]:
shifts1['eventShiftId']

31557

In [320]:
shifts2 = shifts[0][0]

In [321]:
shifts = find_mdw_event['shifts']

dict

In [10]:
source_tbl

outreach_id,action_date,modified_date,created_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,dup,vanid,date_created,sr_load,event_load
9247520,2021-01-24 00:29:34,2021-01-24 00:29:34,2021-01-24 00:29:34,Damaris,Gonzalez,gonzalezmrs34@gmail.com,7138551968,77036,https://organizetexas.org/immigration-2021/,695591,MDW,completed,User hung up phone,1,None,None,None,None
10215645,2021-03-23 17:38:21,2021-03-23 17:38:21,2021-03-23 17:38:21,Justin,Leaca,justin.lecea@gmail.com,8108143304,78229,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,User hung up phone,1,None,None,None,None
10218288,2021-03-23 21:46:21,2021-03-23 21:46:21,2021-03-23 21:46:21,Jessica,Azua,jazua@organizetexas.org,2105773090,78204,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,User hung up phone,1,None,None,None,None
10218193,2021-03-23 21:40:02,2021-03-23 21:40:02,2021-03-23 21:40:02,Jasmine,Estrada,estradaest2009@gmail.com,2104608040,78223,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,User hung up phone,1,None,None,None,None
10215649,2021-03-23 17:38:27,2021-03-23 17:38:27,2021-03-23 17:38:27,Justin,Leaca,justin.lecea@gmail.com,8108143304,78229,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,User hung up phone,1,None,None,None,None


In [12]:
source_tbl.num_rows

16

In [16]:
for ppl in source_tbl:
    if ppl['day_rank'] == 1:
        print('Yes:', ppl['day_rank'])
    else:
        print('No:', ppl['day_rank'])

Yes: 1
Yes: 1
Yes: 1
Yes: 1
No: 3
No: 2
No: 2
Yes: 1
No: 3
Yes: 1
Yes: 1
No: 2
Yes: 1
Yes: 1
Yes: 1
Yes: 1
